In [ ]:
import os
import cv2
import numpy as np
import scipy.linalg
#import visualize as vl
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from mpl_toolkits.mplot3d import Axes3D

Points_2D = []
rows = 9
cols = 4
num_points = rows * cols
Points_3D = np.loadtxt("./data/Point3D.txt", dtype='i', delimiter=' ')

Data_index = 1
Data_number = 2
index = 0
# change input image path here
img = cv2.imread('data/data_1.jpg')
h, w, c = img.shape

def Get_Points_2D(event, x, y, flags, param):
    global Points_2D, index
    if event == cv2.EVENT_LBUTTONDOWN:
        Points_2D += [[x, y]]
    elif event == cv2.EVENT_LBUTTONUP:
        cv2.circle(img, (x, y), 5, (0, 255, 255), -1)
        cv2.imshow('image', img)
        index += 1
        if index >= num_points:
            cv2.destroyAllWindows()
    return Points_2D

def Compute_Projection_Matrix():
    global Points_2D, index
    r, c = Points_3D.shape()
    A = np.zero((r*2, 12))
    for i in range(r):
        A[i*2, 0:3] = Points_3D[i, :]
        A[i*2, 3] = 1
        A[i*2, 8:-1] = -Points_2D[i][0] * Points_3D[i]
        A[i*2, -1]= -Points_2D[i][0]
        A[i*2+1, 7] = 1
        A[i*2+1, 8:-1] = -Points_2D[i][1] * Points_3D[i]
        A[i*2 + 1, -1] = -Points_2D[i][1]

    Eigen_Values, Eigen_Vectors = np.linalg.eig(A.transpose().dot(A))
    Projection_Matrix = Eigen_Vectors[:, np.argmin(Eigen_Values)]  
    Projection_Matrix = Projection_Matrix.reshape((3,4))
    return Projection_Matrix

def Decompose_into_KRT(Projection_Matrix):
    if np.linalg.det(Projection_Matrix[:,:-1])<0:
        Projection_Matrix *= -1
    K, R = scipy.linalg.rq(Projection_Matrix[:,:-1])
    D = np.zeros((3, 3))
    for i in range(3):
        D[i, i] = np.sign(K[i][i])
    K = K.dot(D)
    T = np.linalg.inv(K).dot(Projection_Matrix[:, -1])
    T=T[..., None]
    R = D.dot(R)
    K /= K[-1, -1]
    return K,R,T

def Reproject_into_Points_2D(K, R, T, Points_3D):
    global Points_2D,img
    RT = np.concatenate((R.transpose(), T.transpose()))
    RT = RT.transpose()
    New_Projection_Matrix = K.dot(RT)
    r, c = Points_3D.shape
    Points_3D_4N = np.c_[Points_3D, np.ones(r)]
    Points_2D_3N = New_Projection_Matrix.dot(Points_3D_4N.transpose())
    Points_2D_3N = Points_2D_3N.transpose()
    Points_2D_3N[:, 0] = np.divide(Points_2D_3N[:, 0], Points_2D_3N[:, 2])
    Points_2D_3N[:, 1] = np.divide(Points_2D_3N[:, 1], Points_2D_3N[:, 2])
    RMS=0
    for i in range(r):
        cv2.circle(img, (int(Points_2D[i][0]), int(Points_2D[i][1])), 5, (0, 255, 255), -1)
        cv2.circle(img, (int(Points_2D_3N[i][0]), int(Points_2D_3N[i][1])), 3, (255, 0, 255), -1)
        RMS+=(Points_2D[i][0]-Points_2D_3N[i][0])**2+(Points_2D[i][1]-Points_2D_3N[i][1])**2
    RMS /= r
    RMS=np.sqrt(RMS)
    cv2.imshow('image', img)
    cv2.imwrite('Result' + str(Data_index) + '.jpg', img)
    return Points_2D_3N[:, 0:2], RMS
def Compute_Camera_Position(R, T):
    return R.transpose().dot(T)
    
if __name__ == '__main__':
    #for i in range(2)
    mouse = []
    index = 0
    img_path = 'data/data_' + str(Data_index) +'.jpg'
    img = cv2.imread(img_path)
    h, w, c = img.shape
    
    cv2.namedWindow('image', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('image', w, h)
    cv2.setMouseCallback('image', Get_Points_2D)
    cv2.imshow('image', img)
    cv2.waitKey(0)
    # change output path here
    np.save('hw2-1/Points_2D'+str(Data_index), np.asarray(Points_2D))
    
    Projection_Matrix = Compute_Projection_Matrix()
    np.save('Projection_matrix_'+str(Data_index), np.asarray(Points_2D))
    K, R, T = Decompose_into_KRT(Projection_Matrix)
    np.save('K_matrix_'+str(Data_index), np.asarray(K))
    np.save('R_matrix_'+str(Data_index), np.asarray(R))
    np.save('T_matrix_'+str(Data_index), np.asarray(T))

    cv2.namedWindow('image', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('image', w, h)
    cv2.imshow('image', img)
    Points_2D_3N, RMS = Reproject_into_Points_2D(K, R, T, Points_3D)
    np.save('RMS_'+str(Data_index), np.asarray(RMS))
    cv2.waitKey(0)
